In [1]:
!pip install nltk pandas PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import PyPDF2


In [9]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
from google.colab import files
uploaded = files.upload()


Saving NLP PDF.pdf to NLP PDF.pdf


In [5]:
pdf_file = open(list(uploaded.keys())[0], 'rb')
reader = PyPDF2.PdfReader(pdf_file)

text = ""
for page in reader.pages:
    text += page.extract_text()

text[:1000]


'ByCharlottePerkinsGilman\nIt isveryseldomthat mereordinarypeoplelike\nJohnandmyself secureancestralhallsforthe\nsummer. A colonialmansion, ahereditaryestate, I\nwouldsayahauntedhouse, andreachtheheight\nof romanticfelicity—but that wouldbeaskingtoo\nmuchof fate! StillI willproudlydeclarethat thereis\nsomethingqueerabout it. Else, whyshouldit be\nlet socheaply?Andwhyhavestoodsolong\nuntenanted?Johnlaughsat me, of course, but\noneexpectsthat inmarriage. Johnispracticalin\ntheextreme. Hehasnopatiencewithfaith, an\nintensehorrorof superstition, andhescoffsopenly\nat anytalkof thingsnot tobefelt andseenandput\ndowninfigures. Johnisaphysician, and\nperhaps—(I wouldnot sayit toalivingsoul, of\ncourse, but thisisdeadpaperandagreat relief to\nmymind)—perhapsthat isonereasonI donot get\nwellfaster. Yousee, hedoesnot believeI amsick!\nAndwhat canonedo?If aphysicianof high\nstanding, andone’sownhusband, assuresfriends\nandrelativesthat thereisreallynothingthematter\nwithonebut temporarynervousdep

In [6]:
sentences = [s.strip() for s in text.split('.') if len(s.strip()) > 30]
df = pd.DataFrame(sentences, columns=["Text"])
df.head()


,Text
0,ByCharlottePerkinsGilman\nIt isveryseldomthat ...
1,"A colonialmansion, ahereditaryestate, I\nwould..."
2,"Else, whyshouldit be\nlet socheaply?Andwhyhave..."
3,"Hehasnopatiencewithfaith, an\nintensehorrorof ..."
4,"Johnisaphysician, and\nperhaps—(I wouldnot say..."


In [7]:
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    tokens = word_tokenize(text.lower())
    filtered = [w for w in tokens if w.isalpha() and w not in stop_words]
    stemmed = [stemmer.stem(w) for w in filtered]
    lemmatized = [lemmatizer.lemmatize(w) for w in filtered]
    return tokens, filtered, stemmed, lemmatized


In [11]:
df["tokens"] = df["Text"].apply(lambda x: preprocess(x)[0])
df["no_stopwords"] = df["Text"].apply(lambda x: preprocess(x)[1])
df["stemmed"] = df["Text"].apply(lambda x: preprocess(x)[2])
df["lemmatized"] = df["Text"].apply(lambda x: preprocess(x)[3])

df.head()

,Text,tokens,no_stopwords,stemmed,lemmatized
0,ByCharlottePerkinsGilman\nIt isveryseldomthat ...,"[bycharlotteperkinsgilman, it, isveryseldomtha...","[bycharlotteperkinsgilman, isveryseldomthat, m...","[bycharlotteperkinsgilman, isveryseldomthat, m...","[bycharlotteperkinsgilman, isveryseldomthat, m..."
1,"A colonialmansion, ahereditaryestate, I\nwould...","[a, colonialmansion, ,, ahereditaryestate, ,, ...","[colonialmansion, ahereditaryestate, wouldsaya...","[colonialmans, ahereditaryest, wouldsayahaunte...","[colonialmansion, ahereditaryestate, wouldsaya..."
2,"Else, whyshouldit be\nlet socheaply?Andwhyhave...","[else, ,, whyshouldit, be, let, socheaply, ?, ...","[else, whyshouldit, let, socheaply, andwhyhave...","[els, whyshouldit, let, socheapli, andwhyhaves...","[else, whyshouldit, let, socheaply, andwhyhave..."
3,"Hehasnopatiencewithfaith, an\nintensehorrorof ...","[hehasnopatiencewithfaith, ,, an, intensehorro...","[hehasnopatiencewithfaith, intensehorrorof, su...","[hehasnopatiencewithfaith, intensehorrorof, su...","[hehasnopatiencewithfaith, intensehorrorof, su..."
4,"Johnisaphysician, and\nperhaps—(I wouldnot say...","[johnisaphysician, ,, and, perhaps—, (, i, wou...","[johnisaphysician, wouldnot, sayit, toalivings...","[johnisaphysician, wouldnot, sayit, toalivings...","[johnisaphysician, wouldnot, sayit, toalivings..."


In [12]:
def get_vocab_size(list_of_docs):
    vocab = set()
    for doc in list_of_docs:
        vocab.update(doc)
    return len(vocab)


In [13]:
results = {
    "Original Tokens": get_vocab_size(df["tokens"]),
    "After Stopword Removal": get_vocab_size(df["no_stopwords"]),
    "After Stemming": get_vocab_size(df["stemmed"]),
    "After Lemmatization": get_vocab_size(df["lemmatized"])
}

pd.DataFrame(results, index=["Vocabulary Size"])


,Original Tokens,After Stopword Removal,After Stemming,After Lemmatization
Vocabulary Size,222,169,169,169


In [14]:
df[["Text", "tokens", "no_stopwords", "stemmed", "lemmatized"]].head(3)


,Text,tokens,no_stopwords,stemmed,lemmatized
0,ByCharlottePerkinsGilman\nIt isveryseldomthat ...,"[bycharlotteperkinsgilman, it, isveryseldomtha...","[bycharlotteperkinsgilman, isveryseldomthat, m...","[bycharlotteperkinsgilman, isveryseldomthat, m...","[bycharlotteperkinsgilman, isveryseldomthat, m..."
1,"A colonialmansion, ahereditaryestate, I\nwould...","[a, colonialmansion, ,, ahereditaryestate, ,, ...","[colonialmansion, ahereditaryestate, wouldsaya...","[colonialmans, ahereditaryest, wouldsayahaunte...","[colonialmansion, ahereditaryestate, wouldsaya..."
2,"Else, whyshouldit be\nlet socheaply?Andwhyhave...","[else, ,, whyshouldit, be, let, socheaply, ?, ...","[else, whyshouldit, let, socheaply, andwhyhave...","[els, whyshouldit, let, socheapli, andwhyhaves...","[else, whyshouldit, let, socheaply, andwhyhave..."
